# TP3 : Inférence variationnelle dans le modèle *Latent Dirichlet Allocation*

## G3 SDI - Estimation Avancée

Dans ce dernier TP, on s'intéresse au modèle *Latent Dirichlet Allocation* ([Blei et al. (2003)](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)), un célèbre modèle probabiliste pour données textuelles. Voir le .pdf joint à l'archive pour la description complète du modèle. Dans ce modèle, la loi a posteriori est intractable, et nous utiliserons l'approche variationnelle pour en trouver une approximation.

Using the notations from the .pdf file describing the model, the intractable posterior is :
$$p(\boldsymbol{\beta}, \boldsymbol{\theta} | \mathcal{D}),$$
which we are going to approximate in the following way :
$$\simeq \left[ \prod_{k=1}^K q(\boldsymbol{\beta_k}) \right] \left[ \prod_{d=1}^D q(\boldsymbol{\theta_d}) \right] , $$
with :
* $q(\boldsymbol{\beta_k})$ a Dirichlet distribution (of size V) with variational parameters $[\lambda_{k1}, ...,\lambda_{kV}]$ ;
* $q(\boldsymbol{\theta_d})$ a Dirichlet distribution (of size K) with variational parameters $[\gamma_{d1}, ...,\gamma_{dK}]$.

### Instructions

1. Renommez votre notebook sous la forme `tp3_Nom1_Nom2.ipynb`. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposez votre notebook sur Moodle dans la section prévue à cet effet avant la date limite.

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Compte-rendu écrit par AMORRI Farah, MILANO Olivia, 18/12/2025.
</div>

In [1]:
# Import usual libraries
import numpy as np
from matplotlib import pyplot as plt

# Specific for this lab
from sklearn.decomposition import LatentDirichletAllocation

**Q0.** À quelle difficulté principale se heurterait un algorithme MCMC dans ce contexte ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 1 - Préparation des données

On utilise le dataset `20newsgroups` (voir par exemple [ici](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset)), qui contient des messages postés dans différentes catégories sur un forum dans les années 90.

In [ ]:
import ssl
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer


ssl._create_default_https_context = ssl._create_unverified_context

# Load data
newsgroups = fetch_20newsgroups().data

**Q1.** En LDA, sous quelle forme est représentée un document ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q2.** Créer la matrice des données en utilisant la fonction `CountVectorizer` en utilisant les arguments :
* `max_df` = 0.95 ;
* `max_features` = 1000 ;
* `stop_words` = "english".

Expliquer à quoi correspondent ces 3 arguments.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Création de l'instance CountVectorizer avec les paramètres demandés
tf_vectorizer = CountVectorizer(
    max_df=0.95,
    max_features=1000,
    stop_words="english"
)

# Création de la matrice des données (Matrice Document-Terme)
# X sera une matrice sparse de taille (D x V)
X = tf_vectorizer.fit_transform(newsgroups)

# Récupération des noms des mots (le vocabulaire) pour vérification
feature_names = tf_vectorizer.get_feature_names_out()

print(f"Taille de la matrice : {X.shape}")

Taille de la matrice : (11314, 1000)


<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q3.** Rappeler à quoi correspondent les dimensions de la matrice des données obtenues.

Calculer son pourcentage d'éléments non-nuls. Est-ce surprenant ?

In [ ]:

total_elements = X.shape[0] * X.shape[1]


non_zero_elements = X.nnz


density = (non_zero_elements / total_elements) * 100

print(f"Pourcentage d'éléments non-nuls (Densité) : {density:.4f} %")
print(f"Pourcentage de zéros (Sparsité) : {100 - density:.4f} %")

Pourcentage d'éléments non-nuls (Densité) : 4.4165 %
Pourcentage de zéros (Sparsité) : 95.5835 %


<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 2 - Latent Dirichlet Allocation (LDA)

**Q4.** Entraîner le modèle LDA avec les arguments suivants :
* `n_components` = 8 ;
* `learning_method` = "online" ;
* `max_iter` = 50 ;
* `doc_topic_prior` = 0.1 ;
* `topic_word_prior` = 0.1 ;
* `learning_offset` = 100.

Idem, expliquer à quoi correspondent chacun de ces arguments.

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialisation du modèle avec les hyperparamètres demandés
lda_model = LatentDirichletAllocation(
    n_components=8,
    learning_method="online",
    max_iter=50,
    doc_topic_prior=0.1,
    topic_word_prior=0.1,
    learning_offset=100,
    random_state=42 
)

# Entraînement du modèle sur la matrice des données X (calculée en Q2)
print("Démarrage de l'entraînement (Inférence Variationnelle)...")
lda_model.fit(X)
print("Entraînement terminé.")

Démarrage de l'entraînement (Inférence Variationnelle)...
Entraînement terminé.


<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q5.** Que contient l'attribut `.components` ?

Comment obtenir le MMSE de $\boldsymbol{\beta}$ à partir de cet attribut ? L'implémenter.

In [9]:
import numpy as np


lambda_matrix = lda_model.components_

print(f"Forme de la matrice brute : {lambda_matrix.shape}")
print(f"Somme de la première ligne (avant norm.) : {lambda_matrix[0].sum():.2f}")

# Calcul du MMSE (Espérance de la Dirichlet) -> Normalisation L1 par ligne
# On divise chaque élément par la somme de sa ligne
beta_mmse = lambda_matrix / lambda_matrix.sum(axis=1)[:, np.newaxis]

# Vérification
print("-" * 30)
print(f"Forme de la matrice estimée (beta) : {beta_mmse.shape}")
print(f"Somme de la première ligne (après norm.) : {beta_mmse[0].sum():.2f}") 
# Doit afficher 1.0

Forme de la matrice brute : (8, 1000)
Somme de la première ligne (avant norm.) : 99116.08
------------------------------
Forme de la matrice estimée (beta) : (8, 1000)
Somme de la première ligne (après norm.) : 1.00


<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q6.** À partir de $\hat{\boldsymbol{\beta}}_{MMSE}$, afficher les 10 mots les plus courants par topic. Pouvez-vous interpréter les topics obtenus ?

Si vous souhaitez rajouter certains mots à la liste des stop words pour améliorer l'interprétabilité, utilisez le bloc de code ci-dessous.

In [10]:
def print_top_words(beta_matrix, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(beta_matrix):
        # On trie les indices par probabilité croissante et on prend les 10 derniers
        top_indices = topic.argsort()[:-n_top_words - 1:-1]
        
        # On récupère les mots correspondants
        top_words = [feature_names[i] for i in top_indices]
        
        # Affichage formaté
        print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")

# Récupération du vocabulaire (si ce n'est pas déjà fait)
feature_names = tf_vectorizer.get_feature_names_out()

# Appel de la fonction
print_top_words(beta_mmse, feature_names)

Topic #1: edu, file, uk, mail, ac, information, program, com, university, available
Topic #2: people, government, gun, law, president, don, right, mr, think, state
Topic #3: ax, max, g9v, b8f, a86, 145, pl, 0d, 1d9, 34u
Topic #4: windows, drive, use, card, com, problem, ibm, scsi, dos, mac
Topic #5: god, people, don, think, say, know, just, said, jesus, does
Topic #6: 10, 00, team, game, 1993, 15, 25, 20, apr, year
Topic #7: space, key, gov, nasa, chip, access, encryption, clipper, data, use
Topic #8: edu, com, writes, article, posting, nntp, host, university, just, like


In [ ]:
# To add additional stop words

from sklearn.feature_extraction import text 

my_additional_stop_words = ### YOUR LIST HERE ### #list of strings
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q7.** Que contient `lda.transform(X)` ?

Regarder si les documents sont en général plutôt associés à un ou plusieurs topics.

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 3 - Pour aller plus loin...

Répondre aux questions sans implémenter les solutions discutées.

**Q8.** Quelle est l'influence des paramètres $\alpha$ et $\eta$ ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q9.** Pour aujourd'hui, le nombre de topics $K$ était fixé à l'avance. Comment pourrait-on apprendre ou choisir la valeur de $K$ ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q10.** Quelles sont les principales limites du modèle LDA et quelles potentielles améliorations proposez-vous ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>